In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
	# create model
	model = Sequential()
	model.add(Dense(16, input_dim=13, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, activation='sigmoid', kernel_initializer='uniform'))
	# Compile model
	#optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# load dataset
dataset = pd.read_csv("slim-xAPI-Edu-Data.csv")
# split into input (X) and output (Y) variables

X = dataset


#y = dataset[['Class']]
#students failed if they are in class L.
dataset['Failed'] = np.where(dataset['Class'] == 'L', True, False)
dataset['gender'] = np.where(dataset['gender']=='M',1,0)
dataset['Relation'] = np.where(dataset['Relation']=='Father',1,0)
dataset['ParentAnsweringSurvey'] = np.where(dataset['ParentAnsweringSurvey'] == 'Yes', 1, 0)
dataset['ParentschoolSatisfaction'] = np.where(dataset['ParentschoolSatisfaction'] == 'Yes', 1, 0)
dataset['AbsentMoreThanWeek'] = np.where(dataset['StudentAbsenceDays'] == 'Above-7', 1, 0)
dataset['Semester'] = np.where(dataset['Semester'] == 'F', 1, 0)

#SectionID: 1=A, 2=B, 3=C
#Topic: 1=Arabic, 2=Biology, 3=Chemistry, 4=English, 5=French, 6=Geology, 7=History, 8=IT, 9=Math, 10=Quran, 11=Science, 12=Spanish
#StageID: Lowerlevel = 1, MiddleSchool = 2, HighSchool = 3
X = dataset[['raisedhands', 'VisITedResources', 'SectionID', 'Topic', 'StageID', 'AnnouncementsView', 'Semester', 'Discussion', 'gender', 'Relation', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'AbsentMoreThanWeek']]

Y = dataset[['Failed']]




Using TensorFlow backend.


In [3]:

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=10, epochs=100)
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)

# define the grid search parameters
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
#param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(np.array(X), np.array(Y))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jmloewen/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.251875). Check your callbacks.
  % delta_t_median)


Best: 0.912500 using {'neurons': 1}
0.912500 (0.013502) with: {'neurons': 1}
0.906250 (0.010206) with: {'neurons': 5}
0.900000 (0.005103) with: {'neurons': 10}
0.902083 (0.005893) with: {'neurons': 15}
0.908333 (0.015590) with: {'neurons': 20}
0.910417 (0.007795) with: {'neurons': 25}
0.900000 (0.013502) with: {'neurons': 30}
